### Iteration 2
I plan to complete the DataQuest second walkthrough and implement the complete model. From there, I will try to explore additional avenues to improve the model.
### Train:

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
import re
import operator
titanic = pandas.read_csv("train.csv")

titanic["Age"] = titanic["Age"].fillna(titanic["Age"].median())
titanic.loc[titanic["Sex"] == "male", "Sex"] = 0
titanic.loc[titanic["Sex"] == "female", "Sex"] = 1
titanic["Embarked"] = titanic["Embarked"].fillna("S")
titanic.loc[titanic["Embarked"] == "S", "Embarked"] = 0
titanic.loc[titanic["Embarked"] == "C", "Embarked"] = 1
titanic.loc[titanic["Embarked"] == "Q", "Embarked"] = 2

def get_title(name):
    # Use a regular expression to search for a title.  Titles always consist of capital and lowercase letters, and end with a period.
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

# First, we'll add titles to the test set.
titles = titanic["Name"].apply(get_title)
# We're adding the Dona title to the mapping, because it's in the test set, but not the training set
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2, "Dona": 10}
for k,v in title_mapping.items():
    titles[titles == k] = v
titanic["Title"] = titles
# Check the counts of each unique title.
print(pandas.value_counts(titanic["Title"]))

# Now, we add the family size column.
titanic["FamilySize"] = titanic["SibSp"] + titanic["Parch"]

# Now we can add family ids.
# We'll use the same ids that we did earlier.
#print(family_id_mapping)



# A function to get the title from a name.
family_id_mapping = {}

def get_family_id(row):
    # Find the last name by splitting on a comma
    last_name = row["Name"].split(",")[0]
    # Create the family id
    family_id = "{0}{1}".format(last_name, row["FamilySize"])
    # Look up the id in the mapping
    if family_id not in family_id_mapping:
        if len(family_id_mapping) == 0:
            current_id = 1
        else:
            # Get the maximum id from the mapping and add one to it if we don't have an id
            current_id = (max(family_id_mapping.items(), key=operator.itemgetter(1))[1] + 1)
        family_id_mapping[family_id] = current_id
    return family_id_mapping[family_id]

family_ids = titanic.apply(get_family_id, axis=1)
family_ids[titanic["FamilySize"] < 3] = -1
titanic["FamilyId"] = family_ids
#nameLength = len(titanic_test["Name"])
titanic["NameLength"] = titanic["Name"].apply(lambda x: len(x))



# The algorithms we want to ensemble.
# We're using the more linear predictors for the logistic regression, and everything with the gradient boosting classifier.
algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Title", "FamilyId"]],
    [LogisticRegression(random_state=1), ["Pclass", "Sex", "Fare", "FamilySize", "Title", "Age", "Embarked"]]
]

# Initialize the cross validation folds
kf = KFold(titanic.shape[0], n_folds=3, random_state=1)

predictions = []
for train, test in kf:
    train_target = titanic["Survived"].iloc[train]
    full_test_predictions = []
    # Make predictions for each algorithm on each fold
    for alg, predictors in algorithms:
        # Fit the algorithm on the training data.
        alg.fit(titanic[predictors].iloc[train,:], train_target)
        # Select and predict on the test fold.  
        # The .astype(float) is necessary to convert the dataframe to all floats and avoid an sklearn error.
        test_predictions = alg.predict_proba(titanic[predictors].iloc[test,:].astype(float))[:,1]
        full_test_predictions.append(test_predictions)
    # Use a simple ensembling scheme -- just average the predictions to get the final classification.
    test_predictions = (full_test_predictions[0] + full_test_predictions[1]) / 2
    # Any value over .5 is assumed to be a 1 prediction, and below .5 is a 0 prediction.
    test_predictions[test_predictions <= .5] = 0
    test_predictions[test_predictions > .5] = 1
    predictions.append(test_predictions)

# Put all the predictions together into one array.
predictions = np.concatenate(predictions, axis=0)

# Compute accuracy by comparing to the training data.
accuracy = sum(predictions[predictions == titanic["Survived"]]) / len(predictions)
print(accuracy)

1     517
2     183
3     125
4      40
5       7
6       6
7       5
10      3
8       3
9       2
Name: Title, dtype: int64
0.819304152637


/home/gabriel/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:106: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index


### Test:
After reviewing the Data Quest tutorial, I now understand a lot more about decision tree searches and linear regression. One of the most interesting concepts I ran into while taking the tutorial was the idea that having multiple algorithms that worked better with a diverse range of elements is better than just relying on one algorithm. Covering more bases and making predictions based on that increased coverage makes the model more likely to be correct, and a lot less likely to be overfit to quirks in the data.
The special liberties I took with this model are the removel of all nans in the dataset, in addition to the ones the tutorial prompts, and the subsequent setting of the nans to 0 where applicable. The reasoning behind this was both because I was getting an error and also because it's better practice to ensure the program cleans all the data going into the kaggle test because there might be nans in the test that aren't in the train data.

In [25]:
# First, we'll add titles to the test set.
titanic_test = pandas.read_csv("test.csv")

titanic_test["Age"] = titanic_test["Age"].fillna(titanic_test["Age"].median())
titanic_test.loc[titanic_test["Sex"] == "male", "Sex"] = 0
titanic_test.loc[titanic_test["Sex"] == "female", "Sex"] = 1
titanic_test["Embarked"] = titanic_test["Embarked"].fillna("S")
titanic_test.loc[titanic_test["Embarked"] == "S", "Embarked"] = 0
titanic_test.loc[titanic_test["Embarked"] == "C", "Embarked"] = 1
titanic_test.loc[titanic_test["Embarked"] == "Q", "Embarked"] = 2
titanic_test["Pclass"] = titanic_test["Pclass"].fillna(0)
titanic_test["Fare"] = titanic_test["Fare"].fillna(0)





titles = titanic_test["Name"].apply(get_title)
# We're adding the Dona title to the mapping, because it's in the test set, but not the training set
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2, "Dona": 10}
for k,v in title_mapping.items():
    titles[titles == k] = v
titanic_test["Title"] = titles
# Check the counts of each unique title.
#print(pandas.value_counts(titanic_test["Title"]))

# Now, we add the family size column.
titanic_test["FamilySize"] = titanic_test["SibSp"] + titanic_test["Parch"]

# Now we can add family ids.
# We'll use the same ids that we did earlier.
#print(family_id_mapping)

family_ids = titanic_test.apply(get_family_id, axis=1)
family_ids[titanic_test["FamilySize"] < 3] = -1
titanic_test["FamilyId"] = family_ids
#nameLength = len(titanic_test["Name"])
titanic_test["NameLength"] = titanic_test["Name"].apply(lambda x: len(x))

titanic_test["FamilySize"] = titanic_test["FamilySize"].fillna(0)
titanic_test["Title"] = titanic_test["Title"].fillna("Mr")
titanic_test["FamilyId"] = titanic_test["FamilyId"].fillna(0)

predictors = ["Pclass", "Sex", "Age", "Fare", "Embarked", "FamilySize", "Title", "FamilyId"]

algorithms = [
    [GradientBoostingClassifier(random_state=1, n_estimators=25, max_depth=3), predictors],
    [LogisticRegression(random_state=1), ["Pclass", "Sex", "Fare", "FamilySize", "Title", "Age", "Embarked"]]
]

full_predictions = []
for alg, predictors in algorithms:
    # Fit the algorithm using the full training data.
    alg.fit(titanic[predictors], titanic["Survived"])
    # Predict using the test dataset.  We have to convert all the columns to floats to avoid an error.
    predictions = alg.predict_proba(titanic_test[predictors].astype(float))[:,1]
    full_predictions.append(predictions)

# The gradient boosting classifier generates better predictions, so we weight it higher.
predictions = (full_predictions[0] * 3 + full_predictions[1]) / 4

predictions[predictions <= .5] = 0
predictions[predictions > .5] = 1
predictions = predictions.astype(int)

submission = pandas.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": predictions
    })
submission.to_csv("kaggle6.csv", index=False)

### Review:
Accuracy: 79.904%
Using Ensembling with the old logistic regression algorithm, and a better form of the random forest algorithm, the gradient boosting classifier, the accuracy improved a small amount. Judging by the number of places I moved up in the competition, that small percentage increase is actually pretty close to the best that can be achieved, at least using these algorithms.
The improvement makess sense because using multiple algorithms correctly should be better than just using one optimally. That being said, if this is the average of both of the algorithms, there is a chance that just using the one that scored better this time would make the model more accurate at the cost of losing the adaptibility of the model.
In regard to where I would like to take this model next, I read a lot of the R tutorial. This tutorial focussed more on creation of the algorithms from the ground up, which I enjoyed much more from a learning perspective than I did the Data Quest tutorial, as there were far fewer black boxes. If I could implement some of the more noticeable trends in the way the survival rate went, I could force the model to predict all old women live and all old men die, because thats close enough to true as to make it a reasonable assumption. The rest, I would leave up to the algorithms to decide.